In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
#from sklearn.cross_validation import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc
import os
import seaborn as sns
from datetime import date

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test_bqCt9Pv.csv', 'train.csv', 'sample_submission_24jSKY6.csv']


In [2]:
current_year = date.today().year
current_month = date.today().month


pd.options.display.max_columns = None
pd.options.display.max_rows = None

train_file = '../input/train.csv'
test_file = '../input/test_bqCt9Pv.csv'

train = pd.read_csv(train_file,parse_dates=True)
test = pd.read_csv(test_file,parse_dates=True)

train.shape , test.shape

((233154, 41), (112392, 40))

In [3]:
def grpby_col(df, grpby_col , col, col_name):        
        gp = df.groupby(grpby_col).agg({col:'mean'}).reset_index()
        gp.columns = [grpby_col,col_name]
        df1 = pd.merge(df,gp,on=[grpby_col],how='left')
        df[col_name] = df1[col_name]


def str_len_to_months(x):
        if x is not None:
                year_ = int(x.split('yrs')[0][0].strip())
                month_ = int(x.split('yrs')[1].split('mon')[0].strip())
                t_m = (year_ * 12) + month_
                return t_m
        else:
                return 0

def disbursal_age_months(x):
    x = pd.to_datetime(x)
    year_ = current_year - x.year
    month_ = current_month - x.month
    age = (year_*12) + month_
    return age  

def CNS_SCORE_DESCRIPTION(df):
    df['Very High Risk'] = np.where(df['PERFORM_CNS.SCORE.DESCRIPTION'].str.contains('Very High Risk'),1,0)
    df['Medium Risk'] = np.where(df['PERFORM_CNS.SCORE.DESCRIPTION'].str.contains('Medium Risk'),1,0)
    df['Low Risk'] = np.where(df['PERFORM_CNS.SCORE.DESCRIPTION'].str.contains('Low Risk'),1,0)
    df['Not Scored:'] = np.where(df['PERFORM_CNS.SCORE.DESCRIPTION'].str.contains("No"),1,0)

def df_pre_pros(df):
    df['disb_amnt_dflt_rng'] = np.where(df['disbursed_amount'].between(13000,200000,inclusive = True) ,1,0)
    df['asset_cost_dflt_rng'] = np.where(df['asset_cost'].between(37000,281164,inclusive = True) ,1,0)
    df['supplier_id_dflt_rng'] = np.where(df.supplier_id.isin([3890,13913,13924]) ,1,0)
    df['manufacturer_id_dflt_rng'] = np.where(df.manufacturer_id.isin([86,45]) ,1,0)
    df['Current_pincode_id_dflt_rng'] = np.where(df.Current_pincode_ID.isin([5,17]) ,1,0)
    df['low_risk_pin_range'] = np.where(~df.Current_pincode_ID.isin([5,17]) ,1,0)
    df['Date.of.Birth'] = pd.to_datetime(df['Date.of.Birth'])
    df.loc[(df['Date.of.Birth'].dt.year>2019),['Date.of.Birth']] =  pd.to_datetime('1-1-1985')
    df['age'] = (2019 - df['Date.of.Birth'].dt.year)
    df['age_dflt_rng'] = np.where(df.age.between(23,34,inclusive = True) ,1,0)
    df['Employment_Type'] = [1 if x == 'Self employed' else 0 for x in df['Employment.Type']]
    df.DisbursalDate = pd.to_datetime(df.DisbursalDate)
    df['disbursal_age_months'] = df['DisbursalDate'].apply(disbursal_age_months)
    df['disbursal_age_dflt_rng'] = np.where(df.disbursal_age_months.between(6,8,inclusive = True) ,1,0)
    df['high_risk_rates_4'] = np.where(df.State_ID == 4 ,1,0) # np.where(df.State_ID.isin([4,6,3,13]) ,1,0)
    df['high_risk_rates_6'] = np.where(df.State_ID == 6 ,1,0)
    df['high_risk_rates_3'] = np.where(df.State_ID == 3 ,1,0)
    df['high_risk_rates_13'] = np.where(df.State_ID == 13  ,1,0)
    df['low_risk_rates_13'] = np.where(~df.State_ID.isin([4,6,3,13]) ,1,0)
    df['high_risk_pin_5'] = np.where(df.Current_pincode_ID == 5 ,1,0)
    df['high_risk_pin_17'] = np.where(df.Current_pincode_ID == 17 ,1,0)
    df['AVERAGE_ACCT_AGE'] = df['AVERAGE.ACCT.AGE'].apply(str_len_to_months)
    df['CREDIT_HISTORY_LENGTH'] = df['CREDIT.HISTORY.LENGTH'].apply(str_len_to_months)
    #df['%_of_loan_amnt_apprvd'] = df['disbursed_amount']/df['asset_cost']
    # Group by Pin Code
    grpby_col(df , 'Current_pincode_ID'  , 'disbursed_amount'  , 'pin_dis_amnt')
    grpby_col(df , 'Current_pincode_ID'  , 'ltv'  , 'pin_ltv_amnt')
    grpby_col(df , 'Current_pincode_ID'  , 'asset_cost'  , 'pin_asset_cost')
    grpby_col(df , 'Current_pincode_ID'  , 'PRI.DISBURSED.AMOUNT'  , 'pin_PRI_DISBURSED_AMOUNT')
    grpby_col(df , 'Current_pincode_ID'  , 'PRI.SANCTIONED.AMOUNT'  , 'pin_PRI_SANCTIONED_AMOUNT')
    grpby_col(df , 'Current_pincode_ID'  , 'PRIMARY.INSTAL.AMT'  , 'pin_PRIMARY_INSTAL_AMT')
    # Group by State ID
    grpby_col(df , 'State_ID'  , 'disbursed_amount'  , 'state_dis_amnt')
    grpby_col(df , 'State_ID'  , 'ltv'  , 'state_ltv_amnt')
    grpby_col(df , 'State_ID'  , 'asset_cost'  , 'state_asset_cost')
    grpby_col(df , 'State_ID'  , 'PRI.DISBURSED.AMOUNT'  , 'state_PRI_DISBURSED_AMOUNT')
    grpby_col(df , 'State_ID'  , 'PRI.SANCTIONED.AMOUNT'  , 'state_PRI_SANCTIONED_AMOUNT')
    grpby_col(df , 'State_ID'  , 'PRIMARY.INSTAL.AMT'  , 'state_PRIMARY_INSTAL_AMT')
    df['PRI_CRRNT_BAL_NEG_FLAG'] = np.where(df['PRI.CURRENT.BALANCE']<0 ,1,0)
    df['NO_OF_INQUIRIES_RNG'] = np.where(df['NO.OF_INQUIRIES']<=2 ,1,0)
    df['LST_SIX_MONTHS_DELINQUENT_RNG'] = np.where(df['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS']<=1 ,1,0)
    df['NEW_ACNT_RNG'] = np.where(df['NEW.ACCTS.IN.LAST.SIX.MONTHS']<=1 ,1,0)
    df['High_Risk_CNS_Score_0'] = np.where(df['PERFORM_CNS.SCORE']==0 ,1,0)
    df['High_Risk_CNS_Score_1'] = np.where(df['PERFORM_CNS.SCORE']==300 ,1,0)
    df['Def_EmpId_2546'] = np.where(df['Employee_code_ID']==2546 ,1,0)
    df['PRI.NO.OF.ACCTS_0'] = np.where(df['PRI.NO.OF.ACCTS']==0 ,1,0)
    df['PRI.NO.OF.ACCTS_1'] = np.where(df['PRI.NO.OF.ACCTS']==1 ,1,0)
    df['PRI.NO.OF.ACCTS_2'] = np.where(df['PRI.NO.OF.ACCTS']==2 ,1,0)
    df['PRI.NO.OF.ACCTS_3'] = np.where(df['PRI.NO.OF.ACCTS']==3 ,1,0)
    #PRI.ACTIVE.ACCTS
    df['PRI.ACTIVE.ACCTS_0'] = np.where(df['PRI.ACTIVE.ACCTS']==0 ,1,0)
    df['PRI.ACTIVE.ACCTS_1'] = np.where(df['PRI.ACTIVE.ACCTS']==1 ,1,0)
    df['PRI.ACTIVE.ACCTS_2'] = np.where(df['PRI.ACTIVE.ACCTS']==2 ,1,0)
    #PRI.OVERDUE.ACCTS
    df['PRI.OVERDUE.ACCTS_0'] = np.where(df['PRI.OVERDUE.ACCTS']==0 ,1,0)
    df['PRI.OVERDUE.ACCTS_1'] = np.where(df['PRI.OVERDUE.ACCTS']==1 ,1,0)
    # Primary Remaining Balance 
    df['total_emi_given']=df['PRI.SANCTIONED.AMOUNT']-df['PRI.CURRENT.BALANCE']
    #CREDIT.HISTORY.LENGTH
    df['NO_CREDIT_HISTORY_LENGTH'] = np.where(df['CREDIT_HISTORY_LENGTH']==0 ,1,0)

    #df['DELINQUENT_RNG_NEW_ACCNT_RNG'] = np.where((df['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS']<=1 and df['NEW.ACCTS.IN.LAST.SIX.MONTHS']<=1) ,1,0)

    # drop columns
    df.drop(columns = ['CREDIT.HISTORY.LENGTH','AVERAGE.ACCT.AGE','Date.of.Birth','Employment.Type','DisbursalDate','MobileNo_Avl_Flag','PERFORM_CNS.SCORE.DESCRIPTION'] ,axis = 1, inplace = True)


In [4]:
df_pre_pros(train)
df_pre_pros(test)

features = [c for c in train.columns if c not in ['UniqueID','loan_default']]
target = train['loan_default']

In [5]:
NFOLDS = 3
SEED = 0
NROWS = None

In [6]:
ntrain = train.shape[0]
ntest = test.shape[0]

x_train = train[features]
y_train = train['loan_default']
x_test = test[features]

kf = KFold(n_splits = NFOLDS, shuffle=True, random_state=SEED)

In [7]:
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]

class CatboostWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]
        
class LightGBMWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['feature_fraction_seed'] = seed
        params['bagging_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]


class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

In [8]:
def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [9]:
et_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
    'class_weight' : 'balanced',
}

rf_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2,
    'class_weight' : 'balanced',
}

xgb_params = {
    'seed': 0,
     'gamma' : 2 , 
    'colsample_bytree': 0.8,
    'silent': 1,
    'subsample': 0.8,
    'learning_rate': 0.075,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 5,
    'nrounds': 200 ,
    'scale_pos_weight' : 6
}

catboost_params = {
    'iterations': 200,
    'learning_rate': 0.5,
    'depth': 3,
    'l2_leaf_reg': 40,
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.8,
    'scale_pos_weight': 6,
    'eval_metric': 'AUC',
    'od_type': 'Iter',
    'allow_writing_files': False
}

lightgbm_params = {
    'n_estimators':200,
    'learning_rate':0.1,
    'num_leaves':123,
    'colsample_bytree':0.8,
    'subsample':0.9,
    'max_depth':15,
    'reg_alpha':0.1,
    'reg_lambda':0.1,
    'min_split_gain':0.01,
    'min_child_weight':2 ,
    'is_unbalance' : "True"
}


In [10]:
xg = XgbWrapper(seed=SEED, params=xgb_params)
#et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
#rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatboostWrapper(clf= CatBoostClassifier, seed = SEED, params=catboost_params)
lg = LightGBMWrapper(clf = LGBMClassifier, seed = SEED, params = lightgbm_params)

xg_oof_train, xg_oof_test = get_oof(xg)
#et_oof_train, et_oof_test = get_oof(et)
#rf_oof_train, rf_oof_test = get_oof(rf)
cb_oof_train, cb_oof_test = get_oof(cb)
lgb_oof_train, lgb_oof_test = get_oof(lg)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0:	total: 118ms	remaining: 23.4s
1:	total: 182ms	remaining: 18.1s
2:	total: 244ms	remaining: 16.1s
3:	total: 308ms	remaining: 15.1s
4:	total: 373ms	remaining: 14.6s
5:	total: 441ms	remaining: 14.3s
6:	total: 506ms	remaining: 13.9s
7:	total: 567ms	remaining: 13.6s
8:	total: 630ms	remaining: 13.4s
9:	total: 698ms	remaining: 13.3s
10:	total: 759ms	remaining: 13s
11:	total: 824ms	remaining: 12.9s
12:	total: 886ms	remaining: 12.7s
13:	total: 949ms	remaining: 12.6s
14:	total: 1.01s	remaining: 12.5s
15:	total: 1.08s	remaining: 12.4s
16:	total: 1.14s	remaining: 12.3s
17:	total: 1.21s	remaining: 12.3s
18:	total: 1.27s	remaining: 12.2s
19:	total: 1.34s	remaining: 12.1s
20:	total: 1.41s	remaining: 12s
21:	total: 1.47s	remaining: 11.9s
22:	total: 1.54s	remaining: 11.8s
23:	total: 1.6s	remaining: 11.7s
24:	total: 1.66s	remaining: 11.6s
25:	total: 1.73s	remaining: 11.6s
26:	total: 1.79s	remaining: 11.5s
27:	total: 1.86s	remaining: 11.4s
28:	total: 1.93s	remaining: 11.4s
29:	total: 1.99s	remaining: 1

44:	total: 2.88s	remaining: 9.92s
45:	total: 2.94s	remaining: 9.85s
46:	total: 3s	remaining: 9.77s
47:	total: 3.06s	remaining: 9.7s
48:	total: 3.13s	remaining: 9.64s
49:	total: 3.19s	remaining: 9.56s
50:	total: 3.25s	remaining: 9.5s
51:	total: 3.31s	remaining: 9.43s
52:	total: 3.38s	remaining: 9.38s
53:	total: 3.44s	remaining: 9.31s
54:	total: 3.51s	remaining: 9.25s
55:	total: 3.57s	remaining: 9.18s
56:	total: 3.64s	remaining: 9.12s
57:	total: 3.7s	remaining: 9.05s
58:	total: 3.76s	remaining: 8.98s
59:	total: 3.83s	remaining: 8.93s
60:	total: 3.89s	remaining: 8.87s
61:	total: 3.96s	remaining: 8.81s
62:	total: 4.02s	remaining: 8.74s
63:	total: 4.08s	remaining: 8.68s
64:	total: 4.15s	remaining: 8.62s
65:	total: 4.21s	remaining: 8.56s
66:	total: 4.28s	remaining: 8.5s
67:	total: 4.34s	remaining: 8.43s
68:	total: 4.41s	remaining: 8.37s
69:	total: 4.47s	remaining: 8.3s
70:	total: 4.53s	remaining: 8.24s
71:	total: 4.6s	remaining: 8.17s
72:	total: 4.66s	remaining: 8.11s
73:	total: 4.72s	remain

87:	total: 5.65s	remaining: 7.19s
88:	total: 5.71s	remaining: 7.13s
89:	total: 5.78s	remaining: 7.06s
90:	total: 5.84s	remaining: 7s
91:	total: 5.91s	remaining: 6.93s
92:	total: 5.97s	remaining: 6.87s
93:	total: 6.04s	remaining: 6.81s
94:	total: 6.1s	remaining: 6.75s
95:	total: 6.17s	remaining: 6.69s
96:	total: 6.24s	remaining: 6.63s
97:	total: 6.3s	remaining: 6.56s
98:	total: 6.37s	remaining: 6.5s
99:	total: 6.43s	remaining: 6.43s
100:	total: 6.49s	remaining: 6.36s
101:	total: 6.56s	remaining: 6.3s
102:	total: 6.62s	remaining: 6.24s
103:	total: 6.69s	remaining: 6.17s
104:	total: 6.75s	remaining: 6.11s
105:	total: 6.81s	remaining: 6.04s
106:	total: 6.88s	remaining: 5.98s
107:	total: 6.94s	remaining: 5.91s
108:	total: 7.01s	remaining: 5.85s
109:	total: 7.07s	remaining: 5.79s
110:	total: 7.13s	remaining: 5.72s
111:	total: 7.2s	remaining: 5.66s
112:	total: 7.26s	remaining: 5.59s
113:	total: 7.33s	remaining: 5.53s
114:	total: 7.39s	remaining: 5.46s
115:	total: 7.46s	remaining: 5.4s
116:	to

In [11]:
from sklearn.metrics import roc_auc_score
print("XG-CV: {}".format(roc_auc_score(y_train, xg_oof_train)))
#print("ET-CV: {}".format(roc_auc_score(y_train ,et_oof_train)))
#print("RF-CV: {}".format(roc_auc_score(y_train ,rf_oof_train)))
print("CB-CV: {}".format(roc_auc_score(y_train, cb_oof_train)))
print("CB-CB: {}".format(roc_auc_score(y_train, lgb_oof_train)))

XG-CV: 0.6662393704813734
CB-CV: 0.6635206708681878
CB-CB: 0.6596050109767081


In [12]:
x_train = np.concatenate((xg_oof_train,cb_oof_train , lgb_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, cb_oof_test , lgb_oof_test), axis=1)

print("{},{}".format(x_train.shape, x_test.shape))

(233154, 3),(112392, 3)


In [13]:
logistic_regression = LogisticRegression()
logistic_regression.fit(x_train,y_train)
submission=pd.DataFrame()
submission['UniqueID']=test['UniqueID']
submission['loan_default'] = logistic_regression.predict_proba(x_test)[:,1]
submission.to_csv('submission_nf_stacking_v3.csv',index=False)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [14]:
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='submission_nf_stacking_v3.csv')